In [1]:
import matplotlib.pyplot as plt
# import pfb_functions as pfn
import numpy as np
import pandas as pd
import time

In [2]:
import struct
import numpy as np

def pfn_read(fn):
    with open(fn,'r') as fid:
        x1 = np.fromfile(fid,dtype='>f8', count=1)
        y1 = np.fromfile(fid,dtype='>f8', count=1)
        z1 = np.fromfile(fid,dtype='>f8', count=1)
        nx = np.fromfile(fid,dtype='>i4', count=1)
        ny = np.fromfile(fid,dtype='>i4', count=1)
        nz = np.fromfile(fid,dtype='>i4', count=1)
        dx = np.fromfile(fid,dtype='>f8', count=1)
        dy = np.fromfile(fid,dtype='>f8', count=1)
        dz = np.fromfile(fid,dtype='>f8', count=1)
        ns = np.fromfile(fid,dtype='>i4', count=1)
        var = np.empty((np.int(ny),np.int(nx),np.int(nz)))
        
        for i0 in np.arange(ns):
            ix = np.fromfile(fid,dtype='>i4', count=1)
            iy = np.fromfile(fid,dtype='>i4', count=1)
            iz = np.fromfile(fid,dtype='>i4', count=1)
        
            nnx = np.fromfile(fid,dtype='>i4', count=1)
            nny = np.fromfile(fid,dtype='>i4', count=1)
            nnz = np.fromfile(fid,dtype='>i4', count=1)
       
            rx = np.fromfile(fid,dtype='>i4', count=1) 
            ry = np.fromfile(fid,dtype='>i4', count=1)
            rz = np.fromfile(fid,dtype='>i4', count=1)
            
            for k in np.arange(iz,iz+nnz):
                for i in np.arange(iy,iy+nny):
                    for j in np.arange(ix,ix+nnx):
                        var[i,j,k]=np.fromfile(fid,dtype='>f8', count=1)
        return var    
    

In [3]:
#-----------------------------------------------------------------------------
# Set up run info
#-----------------------------------------------------------------------------
starttime =  1
stoptime  =  168

figure_output_frequency=24

# ,'pfclm1_base40'
case_namelist = ['pfclm1_base1','pfclm1_base40',
                 'pfclm2_base1','pfclm2_base40']
# case_namelist = ['pfclm1_base1']
num_case = len(case_namelist)
data_length = stoptime
total_ET_storage_timeseries =np.zeros((data_length,num_case),dtype=float)
surface_ET_depth_timeseries =np.zeros((data_length,num_case),dtype=float)




In [4]:
#### read pfb file
def ET_compute (file_path,case_index,time_step,ET_storage_timeseries,ET_depth_timeseries):

    if time_step >0:

        fnameevap = ('%s/%s.out.qflx_evap_tot.%05d.pfb' %(file_path,case_index,time_step))
        fnametran = ('%s/%s.out.qflx_tran_veg.%05d.pfb' %(file_path,case_index,time_step))
        print (fnameevap,fnametran)

        evap_mms = pfn_read(fnameevap)
        #print (evap_mms)
        tran_mms = pfn_read(fnametran)
        #print (tran_mms)


        tran_mh =tran_mms *3600 /1000
        evap_mh =evap_mms *3600 /1000

        surface_tran_volume =evap_mh *30 *30
        surface_evap_volume =evap_mh *30 *30

        surface_ET_volume_map=surface_tran_volume+surface_evap_volume

        surface_tran_depth =surface_tran_volume /198 /198 /30 /30
        surface_evap_depth =surface_evap_volume /198 /198 /30 /30

        surface_ET_depth_map=surface_tran_depth+surface_evap_depth

        total_ET_storage = surface_ET_volume_map.sum()
        
        surface_ET_depth = surface_ET_depth_map.mean()
        
#         else:
#             surface_ET_volume_map=np.zeros((168,168,1),dtype=float)
#             surface_ET_depth_map=np.zeros((168,168,1),dtype=float)
#             total_ET_storage=0
#             surface_ET_depth=0
            


#         global total_ET_storage_timeseries
        ET_storage_timeseries[time_step,case_namelist.index(case_index)]= total_ET_storage 
#         print (time_step, case_index, total_ET_storage, ET_storage_timeseries)
        ET_depth_timeseries[time_step,case_namelist.index(case_index)]= surface_ET_depth

        

    return surface_ET_volume_map[:,:,0], surface_ET_depth_map[:,:,0], ET_storage_timeseries, ET_depth_timeseries
#     return surface_ET_volume_map[:,:,0], surface_ET_depth_map[:,:,0]
            
            

In [5]:
def write_1D (outputfile_depth_name,outputfile_volume_name, ET_storage_timeseries, ET_depth_timeseries):
    
#         file = pathlib.Path(outputfile_depth_name)
#         if file.exists ():
#             print ("File exist, continue writing")
#             outF = open(outputfile_depth_name, "a")
            
#             continue
#         else:
#             print ("File not exist, now one was made.")
            np.savetxt(outputfile_volume_name,ET_storage_timeseries ,delimiter=",")
            np.savetxt(outputfile_depth_name,ET_depth_timeseries,delimiter=",")

In [6]:
def plot_ws_2D (time_step,figure_output_frequency,total_ET_depth,total_ET_volume,sfpath,case_name):
    if time_step%figure_output_frequency==0:
        plt.figure(figsize=(10, 7))
        plt.pcolor(total_ET_depth,cmap='jet')
        plt.colorbar()
        plt.savefig('%s/%s_ET_depth_%05d.png'% (sfpath,case_name,time_step))
        plt.close()     
    
        plt.figure(figsize=(10, 7))
        plt.pcolor(total_ET_volume,cmap='jet')
        plt.colorbar()
#         plt.plot
        plt.savefig('%s/%s_ET_volume_%05d.png'% (sfpath,case_name,time_step))
        plt.close()

In [7]:
# Main function
# if __name__ == "__main__":

#     outputfile_depth_name = ('/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/postprocessing/ET_cal/ET_depth.csv')
#     outputfile_volume_name = ('/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/postprocessing/ET_cal/ET_volume.csv')

#     sfpath=('/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/postprocessing/ET_cal/fig/')
#     for case_index in case_namelist:
#         file_path=('/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_%s' %(case_index))
#         for time_step in range(starttime,stoptime):
#             start_time = time.time()
#             print (case_index)
#             print (time_step)
# #             (total_ET_storage,surface_ET_depth, ET_storage_timeseries,ET_depth_timeseries) = ET_compute(file_path,case_index,time_step)
#             (total_ET_storage,surface_ET_depth) = ET_compute(file_path,case_index,time_step)
            
#             plot_ws_2D (time_step,figure_output_frequency,surface_ET_depth,total_ET_storage,sfpath,case_index)

#         # oupt location and file name
        
#             if time_step%stoptime:
#     #                 plot_ws_1D (time_step, total_soilwater_depth,total_saprowater_depth)
#                 write_1D (outputfile_depth_name,outputfile_volume_name,ET_storage_timeseries,ET_depth_timeseries)


In [8]:
from multiprocessing import Pool, TimeoutError
from time import sleep
import os
import pathlib


def main(ET_storage_timeseries,ET_depth_timeseries):
    # start 8 worker processes
    with Pool(processes=100) as pool:
        outputfile_depth_name = ('/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/postprocessing/ET_cal/ET_depth.csv')
        outputfile_volume_name = ('/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/postprocessing/ET_cal/ET_volume.csv')

             
        sfpath=('/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/postprocessing/ET_cal/fig/')
        for case_index in case_namelist:
            file_path=('/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_%s' %(case_index))
            for time_step in range(starttime,stoptime):
                start_time = time.time()
                print (case_index)
                print (time_step)
                res = pool.apply_async(ET_compute, (file_path,case_index,time_step,ET_storage_timeseries,ET_depth_timeseries))
                #(total_ET_storage,surface_ET_depth, ET_storage_timeseries,ET_depth_timeseries) = ET_compute(file_path,case_index,time_step)
                try:
                    (total_ET_storage,surface_ET_depth, ET_storage_timeseries,ET_depth_timeseries)=(res.get(timeout=10))
                    plot_ws_2D (time_step,figure_output_frequency,surface_ET_depth,total_ET_storage,sfpath,case_index)
                except TimeoutError:
                    print(f"timeout error happened for step {time_step}")
                    
                    #ET_storage_timeseries,ET_depth_timeseries = [0.0],[0.0]
                    #ET_storage_timeseries[time_step,case_namelist.index(case_index)]=0.0
                    #ET_depth_timeseries[time_step,case_namelist.index(case_index)]= 0.0
                except (ValueError,NameError):
                    print(f"got value/name error for step {time_step}")
                else:
                    print(f"something is wrong at step {time_step}, but running continues")
                    pass

                        

                    
                
                
        # oupt location and file name
        
#                 if time_step<stoptime:

        write_1D (outputfile_depth_name,outputfile_volume_name, ET_storage_timeseries, ET_depth_timeseries )

main(total_ET_storage_timeseries, surface_ET_depth_timeseries)

/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_evap_tot.00001.pfb /home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_tran_veg.00001.pfb
pfclm1_base1
1
/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_evap_tot.00002.pfb /home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_tran_veg.00002.pfb
something is wrong at step 1, but running continues
pfclm1_base1
2
/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_evap_tot.00003.pfb /home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Co

something is wrong at step 20, but running continues
pfclm1_base1
21
/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_evap_tot.00022.pfb /home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_tran_veg.00022.pfb
something is wrong at step 21, but running continues
pfclm1_base1
22
/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_evap_tot.00023.pfb /home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_tran_veg.00023.pfb
something is wrong at step 22, but running continues
pfclm1_base1
23
/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pf

timeout error happened for step 41
pfclm1_base1
42
/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_evap_tot.00043.pfb /home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_tran_veg.00043.pfb
something is wrong at step 42, but running continues
pfclm1_base1
43
/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_evap_tot.00044.pfb /home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_tran_veg.00044.pfb
something is wrong at step 43, but running continues
pfclm1_base1
44
/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_

something is wrong at step 62, but running continues
pfclm1_base1
63
/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_evap_tot.00064.pfb /home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_tran_veg.00064.pfb
something is wrong at step 63, but running continues
pfclm1_base1
64
/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_evap_tot.00065.pfb /home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_tran_veg.00065.pfb
something is wrong at step 64, but running continues
pfclm1_base1
65
/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pf

something is wrong at step 83, but running continues
pfclm1_base1
84
/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_evap_tot.00085.pfb /home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_tran_veg.00085.pfb
something is wrong at step 84, but running continues
pfclm1_base1
85
something is wrong at step 85, but running continues
pfclm1_base1
86
/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_evap_tot.00086.pfb /home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_tran_veg.00086.pfb
/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pf

/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_evap_tot.00105.pfb /home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_tran_veg.00105.pfb
something is wrong at step 104, but running continues
pfclm1_base1
105
/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_evap_tot.00106.pfb /home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_tran_veg.00106.pfb
something is wrong at step 105, but running continues
pfclm1_base1
106
/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_pfclm1_base1/pfclm1_base1.out.qflx_evap_tot.00107.pfb /home/chaoche

MaybeEncodingError: Error sending result: '<multiprocessing.pool.ExceptionWithTraceback object at 0x7fda09fdd590>'. Reason: 'PicklingError("Can't pickle <class 'numpy.core._exceptions.MemoryError'>: attribute lookup MemoryError on numpy.core._exceptions failed")'